In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import keras

with io.open('Swami.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


/Users/shivanikohli/Documents/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


corpus length: 637135


In [2]:
# chars = sorted(list(set(text)))
# words = set(open('Swami.txt').read().lower().split())

# print("chars:",type(chars))
# print("words",type(words))
# print("total number of unique words",len(words))
# print("total number of unique chars", len(chars))


# word_indices = dict((c, i) for i, c in enumerate(words))
# indices_word = dict((i, c) for i, c in enumerate(words))

# print("word_indices", type(word_indices), "length:",len(word_indices) )
# print("indices_words", type(indices_word), "length", len(indices_word))
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 102


In [5]:
# maxlen = 30
# step = 3
# print("maxlen:",maxlen,"step:", step)
# sentences = []
# next_words = []
# next_words= []
# sentences1 = []
# list_words = []

# sentences2=[]
# list_words=text.lower().split()
maxlen = 40
step = 3
sentences = []
next_chars = []

In [6]:
# for i in range(0,len(list_words)-maxlen, step):
#     sentences2 = ' '.join(list_words[i: i + maxlen])
#     sentences.append(sentences2)
#     next_words.append((list_words[i + maxlen]))
# print('nb sequences(length of sentences):', len(sentences))
# print("length of next_word",len(next_words))

# print('Vectorization...')
# X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
# y = np.zeros((len(sentences), len(words)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#     for t, word in enumerate(sentence.split()):
#         #print(i,t,word)
#         X[i, t, word_indices[word]] = 1
#     y[i, word_indices[next_words[i]]] = 1
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

nb sequences: 212365
Vectorization...


In [27]:
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


Build model...


In [28]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2,0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

#         for i in range(200):
#             x_pred = np.zeros((1, maxlen, len(words)))
#             for t, word in enumerate(sentence):
#                 x_pred[0, t, char_indices[word]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_word = indices_words[next_index]

#             generated += next_word
#             sentence = sentence[1:] + next_word

#             sys.stdout.write(next_word)
#             sys.stdout.flush()
#         print()
        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


In [29]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y, #Big X for words
          batch_size=1024,
          epochs=25,
          callbacks=[print_callback])
# model.save_weights('Weights/')

Epoch 1/25
212365/212365 [==============================] - 78s 368us/step - loss: 2.3905

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "y picks her up in his arms. he crosses t"
y picks her up in his arms. he crosses to the for and and and to tounge to the tout of the mast of the mattions of the to the somations and and and and and to the the to sumation and and to the tout in the to tour are and to to tour and and to the mand to the simation and and and and and and mant to the to the tout and be tout of the tout in the and and and to the soment of and to that in the mathing and and and to the to to sumant and 
----- diversity: 0.5
----- Generating with seed: "y picks her up in his arms. he crosses t"
y picks her up in his arms. he crosses to you to the moming out the tant and want out in to porection, beense suallust of the tout in the have in the tion ale wound one and abe somputing ind wat gounged and man to inanon is soud in the enouns on o

s way, the fault is not really yours. the votts going. they hon ut, up ox, clarring to reveling to be ancinging craplening ofmorioeflunce and relone togtrab, becaubeling your even fin cofping be that beabitite sonter onlect bring itnous praden. lives how egating cruint-bolity, that is karma butter songeribled yourselfusiony.
monct, evenoty rag? 
gaing they ghibgraine. hip seggen, you but gration. they can holds to rice ains. you resuct 
Epoch 6/25
212365/212365 [==============================] - 80s 378us/step - loss: 1.6230

----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "s in english.
god is the finest artist, "
s in english.
god is the finest artist, the can to the bearner to be the said to be the the experience to meditations and the such be the some all to be the more to deting to be the do not to be a grant to be the see to come to deting to deting to do not to deting to deting to be the to be the sign for the first to be the becises and the

d has now impacted someone else you may to make it is not allow of the end or played to the restless and the spill on tree, however, they really like the one change or stread everything they behave the new agated that someone one your what they are seek of to drompers and the promide in a people with a self-dories allowe and what is a religion. you want to so not are not make the same of the spile and behog porting that you have the wan
----- diversity: 1.0
----- Generating with seed: "d has now impacted someone else you may "
d has now impacted someone else you may gooted you will that mas will that lust happen upor it your one colonely certame there, they also, eady drobing you in the religiores of refictice of belies, rextsion or no concentrats that had and of only fasth are fine res, “wand to tells on dabteess now. your spile too winds often thousbeent with your pick passioned point. your prenulice of comperatement outsident, a sippent. such moritye: an c
Epoch 11/25
212365/212365 

negative. so, for your own good, be positive and the state of the restable and such a state of the restless of the same and a conscious and state of the reading to continue and the four do not see start to like the last to lead and the read and the read and something the consciousness and such a promides and see state of the man feel and the state of the man and state of the consciousness and the read a second and the consciousness and 
----- diversity: 0.5
----- Generating with seed: "negative. so, for your own good, be posi"
negative. so, for your own good, be positive your partner that the one will at concentice. come and state we have the world like and love, you cannot bething a state of your present and the rest from you are no murt to leader to practice of arriet and searing and we can did and who to a down and state of the ready will a start to get mentally and the experience and that the was a method want to the to deserute and this life in the wore 
----- diversity: 1.0
-----

the most. cat and trackic simply, the needs complaying is even stummed with your bream appure the formous blurning te is more than one get believe to be typecal foe will noursen the back on you causother had “part or, feesed, the while yourmind no offer getrs expectat
Epoch 20/25
212365/212365 [==============================] - 78s 366us/step - loss: 1.4333

----- Generating text after Epoch: 19
----- diversity: 0.2
----- Generating with seed: "na sajjana payi ava, virtues accumulate "
na sajjana payi ava, virtues accumulate of the prodicual and start to the positive emotions and the passion to make the truth of the one and the past that is a profections and in the prodicual and when you are the more the success and subling to the profections and start the start the world is a profections and start the profection of the past of the passion of the possions and start the subtle and the promice of the most and the prosic
----- diversity: 0.5
----- Generating with seed: "na sajjana payi av

 a state of mind, albeit not the desirable an a going to a life in the that the right with the past to to fore it is in the own transformation. i am will find over the seem it because you have to a sense of mind for the ralling for your donkey an in a securian in the emotions to a quiet to the master the actions of the have a past of end of else of my back an a religion of what uncertain of the choider going to such as if you are but th
----- diversity: 1.0
----- Generating with seed: " a state of mind, albeit not the desirab"
 a state of mind, albeit not the desirable in the ederging refall is outside a liff a cannet. howeving the fores.
a of the manthen from. the day upon with simple a gives, the infinitif an elture: with as sir. all impondin on your own prodid, an ant. wealthin play and body and alsol seeking of the one want of unbaging the more for i can an om. your gedition and contrant will not even buain viegit, to comes vay to place of the channer.
i’
Epoch 25/25
212365/212365 